In [4]:
pip install anytree

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install anytree --user

Note: you may need to restart the kernel to use updated packages.


In [2]:
from anytree import Node, NodeMixin, RenderTree

In [3]:
# Memperluas setiap kelas python menjadi simpul pohon

class MyBaseClass(object):
    foo = 4
    
class TreeNode(MyBaseClass, NodeMixin):
    def __init__(self, subset, purity, glcm_index=0, glcm_median=0, dominant_class='', parent=None, children=None):
        super(TreeNode, self).__init__()
        self.subset = subset
        self.purity = purity
        self.glcm_index = glcm_index
        self.glcm_median = glcm_median
        self.dominant_class = dominant_class
        if(not children == None):
            self.children = children

In [55]:
from PIL import Image
from skimage.feature import greycomatrix, greycoprops
import glob
import math
import matplotlib.pyplot as plt
import os
import random
import csv
import pandas as pd

In [56]:
model = {}#dictionary {'cap' => {(contrast, correllation, energy, homogenity),...}
class_names = []                        # 'tulis' => {(contrast, correllation, energy, homogenity),..}
map_8bit_to_3bit = [i // 32 for i in range (256)] # 1 - 255 -> 0 -  7
prediksi=[]

In [57]:
# Function (function yang digunakan sama seperti KNN kemarin)

def load_img(img_path):
    return Image.open(img_path).convert('L')

def get_img_size(img):
    return img.size

def print_img(img):
    plt.imshow(img, cmap='gray')
    
def get_resized_img(img, dimension):
    return img.resize(dimension)

def get_img_colors(img):
    return list(img.getdata())

def get_3bit_img_colors(img):
    img_colors = get_img_colors(img)
    
    loop_count = 0
    for img_color in img_colors:
        img_colors[loop_count] = map_8bit_to_3bit[img_color]
        
        loop_count += 1
        
    return img_colors

def get_img_matrix(img_colors):
    img_matrix = []
    
    loop_count = 0
    img_square_dimension = int(math.sqrt(len(img_colors)))
    
    for row in range(img_square_dimension):
        temp_row = []
        for col in range(img_square_dimension):
            temp_row.append(img_colors[loop_count])
            
            loop_count += 1
        img_matrix.append(temp_row)
        
    return img_matrix

def get_img_features(img):
    img_3bit_colors = get_3bit_img_colors(img)
    img_matrix = get_img_matrix(img_3bit_colors)
    
    glcm_matrix = greycomatrix(img_matrix, distances=[1], angles=[0], levels=12, symmetric=False, normed=False)
    glcm_components = ['contrast', 'correlation', 'energy', 'homogeneity', 'ASM', 'dissimilarity']
    
    img_features = []
    for glcm_component in glcm_components:
        img_features.append(greycoprops(glcm_matrix, glcm_component)[0][0])
        
    return tuple(img_features)


In [70]:
# Modelling 

def init_model():
    global model
    global class_names
    
    model = []
    class_names = []
    
def load_class_names(training_path):
    global class_names
    
    class_names = [class_name for class_name in os.listdir(training_path)]
            
def insert_img_features_into_model(class_name, img_features):
    global model

    row_tuple = (class_name,) + img_features
    model.append(row_tuple)
        
def load_preprocessed_img(img_path, dimension=(128, 128)):
    img = load_img(img_path)
    img = get_resized_img(img, dimension)
    
    return img

def train(training_path, img_type='*.jpg'):
    print('Training...')
    for class_name in class_names:
        img_paths = glob.glob(training_path + class_name + '/' + img_type)
        
        for img_path in img_paths:            
            img = load_preprocessed_img(img_path)
            img_features = get_img_features(img)

            insert_img_features_into_model(class_name, img_features)
            
    print('--> Done')
    
def get_splitted_model(sample_rate=0.8):
    sample_model = []
    test_sample_model = []
    
    for row in model:
        random_splitter = random.uniform(0, 1)
        
        if(random_splitter <= sample_rate):
            sample_model.append(row)
        else:
            test_sample_model.append(row)
            
    return sample_model, test_sample_model

def get_subset_purity(subset, minimum_purity=0.8): # Bagian untuk menentukan kemurnian dan bkan kemurnian
    if(len(subset) > 0):        
        total_class = len(class_names)
        total_class_score = len(subset)
        subset_class_scores = [0 for i in range(total_class)]
        
        for row in subset:
            row_class_name = row[0]
            subset_class_scores[class_names.index(row_class_name)] += 1
            
        subset_purity = 0
        for class_index in range(total_class):
            subset_purity += (subset_class_scores[class_index] / total_class_score) ** 2
            
        dominant_class = ''
        if(subset_purity >= minimum_purity):
            maximum_class_scores = max(subset_class_scores) # Pada class yang ada, sistem akan mengidentifikasi nilai maks yang berada di dalam tabel
            dominant_class_index = subset_class_scores.index(maximum_class_scores) # bagian yang memiliki nilai tertinggi
            dominant_class = class_names[dominant_class_index]
            
        return (subset_purity, dominant_class)
    else:
        return (0, '')
    
def get_tree_seed(subset, minimum_purity=0.8):
    tree = TreeNode(subset, get_subset_purity(subset, minimum_purity)[0])
    
    return tree

def get_decision_tree(decision_tree, minimum_purity=0.8, n_branch=2, glcm_length=6):
    if(decision_tree.purity >= minimum_purity or len(decision_tree.subset) == 0):
        return decision_tree
    else:
        subset = decision_tree.subset
        
        branch_future_subset_purities = [[glcm_index, 0] for glcm_index in range(glcm_length)]
        # Kemurnian pada bagian parent yang mempunyai nilai X
        branch_glcm_medians           = [[0 for glcm_index in range(glcm_length)] for i in range(n_branch)]
        # Nilai tengah atau GLCM terakhir setelah diurutkan berdasarkan parent x
        branch_subsets                = [[[] for glcm_index in range(glcm_length)] for i in range(n_branch)]
        # cabang bagian dari parent X
        branch_purities               = [[0 for glcm_index in range(glcm_length)] for i in range(n_branch)]
        # kemurnian parent X
        branch_dominants              = [['' for glcm_index in range(glcm_length)] for i in range(n_branch)]
        # kelas (cap/tulis) yang mendominasi si branch subset
        
        for glcm_index in range(glcm_length):
            sorted_subset = sorted(subset, key=lambda x: x[glcm_index + 1])
            sorted_subset_length = len(sorted_subset)
            # sistem akan melakukan sortir sebelum mencari nilai tengah sebagai inisiasi awal melakukan percabangan
            
            if(sorted_subset_length < n_branch):
                n_branch = sorted_subset_length
                
            row_per_branch = int(sorted_subset_length / n_branch)
            branch_constraint = [row_per_branch * i for i in range(n_branch + 1)]
            branch_constraint[-1] = sorted_subset_length
            
            for branch_index in range(n_branch):
                branch_subsets[branch_index][glcm_index] = sorted_subset[branch_constraint[branch_index]:branch_constraint[branch_index + 1]]
                branch_glcm_medians[branch_index][glcm_index] = branch_subsets[branch_index][glcm_index][-1][glcm_index + 1]
                branch_purity_and_dominant = get_subset_purity(branch_subsets[branch_index][glcm_index], minimum_purity)
                branch_purities[branch_index][glcm_index] = branch_purity_and_dominant[0]
                branch_dominants[branch_index][glcm_index] = branch_purity_and_dominant[1]
                
                if(branch_purities[branch_index][glcm_index] >= minimum_purity):
                    branch_future_subset_purities[glcm_index][1] += 1
                    
        sorted_branch_future_subset_purities = sorted(branch_future_subset_purities, key= lambda x: -x[1])
        glcm_index = sorted_branch_future_subset_purities[0][0]
            
        for branch_index in range(n_branch):
            branch_subset = branch_subsets[branch_index][glcm_index]
            branch_purity = branch_purities[branch_index][glcm_index]
            branch_glcm_median = branch_glcm_medians[branch_index][glcm_index]
            branch_dominant = branch_dominants[branch_index][glcm_index]
                
            decision_tree_branch = TreeNode(branch_subset, branch_purity, glcm_index, branch_glcm_median, branch_dominant)
            decision_tree_branch = get_decision_tree(decision_tree_branch, minimum_purity, n_branch)
            if(not decision_tree_branch == None):
                decision_tree_branch.parent = decision_tree
            
        return decision_tree
    
def get_random_forest(n_decision_tree, n_decision_tree_branch, sample_rate=0.8, minimum_purity=0.8, epochs=1):
    random_forest = []
    
    for dtree_index in range(n_decision_tree):
        print('Creating Decision Tree ' + str(dtree_index + 1) + ' ...')
        sample_model = []
        test_sample_model = []
        for epoch in range(epochs):
            new_sample_model, new_test_sample_model = get_splitted_model(sample_rate)
            sample_model += new_sample_model
            test_sample_model += new_test_sample_model
            dtree_seed = get_tree_seed(sample_model, minimum_purity)
            dtree = get_decision_tree(dtree_seed, minimum_purity, n_decision_tree_branch)
            test(dtree, test_sample_model, minimum_purity)
        random_forest.append(dtree)
        print('Added Decision Tree to Random Forest!')
        print()
        
    return random_forest   
    
def print_decision_tree(decision_tree):
    print(RenderTree(decision_tree))
    
def print_random_forest(random_forest):
    random_forest_length = len(random_forest)
    for dtree_index in range(random_forest_length):
        print('Decision Tree ' + str(dtree_index + 1))
        print(RenderTree(random_forest[dtree_index]))
        print()
        
def get_img_features_class_with_decision_tree(img_features, decision_tree, minimum_purity=0.8):
    temp_decision_tree = decision_tree
    
    while(temp_decision_tree.purity < minimum_purity and not temp_decision_tree.children == ()):
        childrens = temp_decision_tree.children
        total_childs = len(childrens)
        
        is_glcm_median_not_available = True
        child_index = 0
        for children in childrens:
            glcm_index = children.glcm_index
            glcm_median = children.glcm_median
            
            if(img_features[glcm_index] <= glcm_median):
                is_glcm_median_not_available = False
                break
                
            child_index += 1
                
        if(child_index >= total_childs):
            temp_decision_tree = childrens[-1]
        else:
            temp_decision_tree = childrens[child_index]
            
    img_class_name = temp_decision_tree.dominant_class
    
    return img_class_name

def get_img_class_with_decision_tree(img, decision_tree, minimum_purity=0.8):
    img_features = get_img_features(img)
            
    img_class_name = get_img_features_class_with_decision_tree(img_features, decision_tree, minimum_purity)
    
    return img_class_name

def get_img_features_class(img_features, random_forest, minimum_purity=0.8):
    guessed_img_class_name = []
    for dtree in random_forest:
        local_img_class_name = get_img_features_class_with_decision_tree(img_features, dtree, minimum_purity)
        guessed_img_class_name.append(local_img_class_name)
        
    img_class_name = max(set(guessed_img_class_name), key=guessed_img_class_name.count)
    
    return img_class_name

def get_img_class(img, random_forest, minimum_purity=0.8):
    img_features = get_img_features(img)
    
    img_class_name = get_img_features_class(img_features, random_forest, minimum_purity)
    
    return img_class_name

def test(decision_tree, test_sample_model, minimum_purity=0.8):
    print('Testing...')
    total_correct_answer = 0
    total_guess = 0
    
    for row in test_sample_model:
        expected_img_class_name = row[0]
        test_img_features = row[1:]
        test_img_class_name = get_img_features_class_with_decision_tree(test_img_features, decision_tree, minimum_purity)
            
        if(expected_img_class_name == test_img_class_name):
            total_correct_answer += 1
                
        total_guess += 1
            
    accuracy = (total_correct_answer / total_guess) * 100
    
    print('--> Done, accuracy = ' + str(accuracy) + ' %')
    
    return accuracy
        

def validate(validation_path, random_forest, epochs, iteration, minimum_purity=0.8, img_type='*.jpg'):
    validation_precisions = []
    validation_recalls = []
    validation_f1_scores = []
    
    class_names_length = len(class_names)
    true_guess = [0 for i in range(class_names_length)]
    false_guess = [0 for i in range (class_names_length)]
    
    
    #falsepositive_tulis, falsenegatif_tulis, truepositive_tulis, truenegatif_tulis = 0,0,0,0
    #falsepoditive_cap, falsenegatif_cap, truepositif_cap, truenegatif_cap = 0,0,0,0
    print('Validating...')
    total_correct_answer = 0
    total_guess = 0
    
    for class_name in class_names:
        img_paths = glob.glob(validation_path + class_name + '/' + img_type)
        for img_path in img_paths:
            img = load_preprocessed_img(img_path)
            img_class_name = get_img_class(img, random_forest, minimum_purity)
            
            if(class_name == img_class_name):
                true_guess[class_names.index (img_class_name)] +=1
            else :
                false_guess[class_names.index(img_class_name)] +=1 
    total_correct_answer = 0
    total_guess = 0
    for class_id in range(class_names_length):
        total_correct_answer += true_guess[class_id]
        total_guess += true_guess[class_id] + false_guess[class_id]

        class_precision = round(true_guess[class_id] / (true_guess[class_id] + sum(false_guess[0:class_id] + false_guess[class_id+1:])), 4)
        class_recall    = round(true_guess[class_id] / (true_guess[class_id] + false_guess[class_id]), 4)
        class_f1_score  = round((2 * class_precision * class_recall) / (class_precision + class_recall), 4) 
        
        validation_precisions.append(class_precision)
        validation_recalls.append(class_recall)
        validation_f1_scores.append(class_f1_score)
        
    validation_accuracy  = round(total_correct_answer / total_guess, 4)
    validation_loss      = 1 - validation_accuracy
    validation_precision = round(sum(validation_precisions) / len(validation_precisions), 4)
    validation_recall    = round(sum(validation_recalls) / len(validation_recalls), 4)
    validation_f1_score  = round(sum(validation_f1_scores) / len(validation_f1_scores), 4)
        
    validation_metrics = pd.DataFrame(columns=['class', 'precision', 'recall', 'f1-score'])
    for class_id in range(len(class_names)):
        validation_metrics.loc[len(validation_metrics)] = [class_names[class_id], validation_precisions[class_id], validation_recalls[class_id], validation_f1_scores[class_id]]
    validation_metrics.loc[len(validation_metrics)] = ['[avg]', validation_precision, validation_recall, validation_f1_score]
    print (validation_metrics)
    
    save_validation_metrics(validation_metrics, epochs, iteration)
    return validation_accuracy

def save_validation_metrics(validation_metrics, epochs, iteration):
    validation_metrics_path = 'validation_metrics/'
    if(not os.path.exists(validation_metrics_path)):
        os.mkdir(validation_metrics_path)
    validation_metrics_path = validation_metrics_path + '_' +  str(epochs) + '_' + str(iteration) + '.csv'
    validation_metrics.to_csv(validation_metrics_path)
        
def peformance(epochs):
    
    prediksi=[]
    falsepositive_tulis, falsenegatif_tulis, truepositive_tulis, truenegatif_tulis = 0,0,0,0
    falsepoditive_cap, falsenegatif_cap, truepositif_cap, truenegatif_cap = 0,0,0,0

    for class_name in class_names:
        for i in range(256): 
            if(prediksi[i] == "cap"):
                if(class_name == "cap"):
                    truepositive_cap+=1
                else:
                    truenegatif_tulis+=1
            else:
                if(class_name == "cap"):
                    falsepositive_cap+=1
                else:
                    falsenegatif_tulis+=1

        for j in range(i+1, 256):
            if(prediksi[j] == "tulis"):
                if(class_name == "cap"):
                    truenegatif_cap+=1
                else:
                    truepositive_tulis+=1
            else:
                if(class_name == "cap"):
                    falsenegatif_cap+=1
                else:
                    falsepositive_tulis+=1

    recall_tulis = round(((truepositive_tulis / (truepositive_tulis + falsenegatif_tulis))*100),4)
    precission_tulis = round(((truepositive_tulis / (truepositive_tulis + falsepositive_tulis))*100),4)
    f1_score_tulis =  2 * (recall_tulis*precission_tulis) / (recall_tulis + precission_tulis)

    recall_cap = round(((truepositive_cap / (truepositive_cap + falsenegatif_cap))*100),4)
    precission_cap = round(((truepositive_cap / (truepositive_cap + falsepositive_cap))*100),4)
    f1_score_cap=  2 * (recall_cap*precission_cap) / (recall_cap + precission_cap)
    
    recall = (recall_cap+recall_tulis)/2
    precission = (precission_cap + precission_tulis)/2
    f1_score = (f1_score_cap + f1_score_tulis)/2
    
 
    csvRow = ["class", "Recall", "Precission", "F1-Score"]
    csvfile = "Peformance/10_iterasi/"+str(epoch+1)+".csv"
    with open(csvfile, "w") as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(csvRow)
        wr.writerow(['cap', recall_cap, precission_cap, f1_score_cap])
        wr.writerow(['tulis', recall_tulis, precission_tulis, f1_score_tulis])
        wr.writerow(['avg/total', recall, precission, f1_score])

In [71]:
# Defining paths
root_path       = './'
training_path   = root_path + 'training/'
validation_path = root_path + 'validasi/'
test_path       = root_path + 'test/'

# Props
img_type = '*.jpg'
n_decision_tree = 5
n_branch_decision_tree = 4
sample_rate = 0.80
minimum_purity = 0.95
epochs = [20, 30, 50]
iterations = [5 for i in range(len(epochs))]

# Init
init_model()
load_class_names(training_path)

for i in range (len(epochs)):
    for iters in range (iterations[i]):
        train(training_path, img_type)
        rforest = get_random_forest(n_decision_tree, n_branch_decision_tree, sample_rate, minimum_purity, epochs[i])
        model_accuracy = validate(validation_path, rforest, epochs[i], iters, minimum_purity, img_type)

print(model_accuracy)

Training...
--> Done
Creating Decision Tree 1 ...
Testing...
--> Done, accuracy = 87.77777777777777 %
Testing...
--> Done, accuracy = 96.5 %
Testing...
--> Done, accuracy = 98.63013698630137 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.61389961389962 %
Testing...
--> Done, accuracy = 99.19871794871796 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.68944099378882 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.6661101836394 %
Testing...
--> Done, accuracy = 99.69719909159728 %
Testing...
--> Done, accuracy = 99.65156794425087 %
Testing...
--> Done, accuracy = 99.41634241245137 %
Testing...
--> Done, accuracy = 99.45288753799392 %
Testing...
--> Done, accuracy = 99.25586720091586 %
Testing...
--> Done, accuracy = 99.57424161788185 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 100.0 %
Added De

Testing...
--> Done, accuracy = 97.59615384615384 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.5507637017071 %
Testing...
--> Done, accuracy = 99.9252615844544 %
Testing...
--> Done, accuracy = 99.93476842791911 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.89701338825952 %
Testing...
--> Done, accuracy = 99.90774907749078 %
Testing...
--> Done, accuracy = 99.79009235936188 %
Testing...
--> Done, accuracy = 99.29851909586905 %
Testing...
--> Done, accuracy = 99.6065808297568 %
Testing...
--> Done, accuracy = 99.63552021206097 %
Testing...
--> Done, accuracy = 99.2622194897018 %
Testing...
--> Done, accuracy = 99.82568274259151 %
Testing...
--> Done, accuracy = 99.25885259401592 %
Testing...
--> Done, accuracy = 99.63617463617463 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.76646426903316 %
Added Decision Tree to R

Testing...
--> Done, accuracy = 99.91160872127284 %
Testing...
--> Done, accuracy = 99.60886571056062 %
Testing...
--> Done, accuracy = 99.55357142857143 %
Testing...
--> Done, accuracy = 99.72114972114973 %
Testing...
--> Done, accuracy = 99.64664310954063 %
Testing...
--> Done, accuracy = 99.5493871665465 %
Testing...
--> Done, accuracy = 99.66482319423497 %
Testing...
--> Done, accuracy = 99.92228784581908 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.83171521035598 %
Testing...
--> Done, accuracy = 99.86479842674532 %
Testing...
--> Done, accuracy = 99.66109617856725 %
Added Decision Tree to Random Forest!

Creating Decision Tree 2 ...
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.7287872917474 %
Testing...
--> Done, accuracy 

Testing...
--> Done, accuracy = 99.6303317535545 %
Added Decision Tree to Random Forest!

Creating Decision Tree 4 ...
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.72247918593895 %
Testing...
--> Done, accuracy = 99.81527093596058 %
Testing...
--> Done, accuracy = 99.81386691484411 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.62168978562421 %
Testing...
--> Done, accuracy = 99.05533063427801 %
Testing...
--> Done, accuracy = 99.76470588235294 %
Testing...
--> Done, accuracy = 99.53888073779082 %
Testing...
--> Done, accuracy = 99.02145276627776 %
Testing...
--> Done, accuracy = 99.79657569079505 %
Testing...
--> Done, accuracy = 99.10049627791562 %
Testing...
--> Done, accuracy = 99.69870875179339 %
Testing...
--> Done, accuracy = 99.73379475575669 %
Testing...
--> Done, accuracy = 99.35491874457263 %
Testing...
--> Done, accuracy = 99.3828598043782 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 10

Testing...
--> Done, accuracy = 98.85285682770792 %
Testing...
--> Done, accuracy = 99.05362776025235 %
Testing...
--> Done, accuracy = 98.84009386523634 %
Testing...
--> Done, accuracy = 99.0288764550775 %
Testing...
--> Done, accuracy = 99.25935069744476 %
Testing...
--> Done, accuracy = 98.73417721518987 %
Testing...
--> Done, accuracy = 99.20916905444126 %
Testing...
--> Done, accuracy = 99.11519106342975 %
Testing...
--> Done, accuracy = 99.50907150480256 %
Testing...
--> Done, accuracy = 98.9060323030084 %
Added Decision Tree to Random Forest!

Creating Decision Tree 5 ...
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.77494373593399 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.42374183634269 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.74166881942651 %
Testing...
--> Done, accuracy = 99.82355535950596 %
Testing...
--> Done, accuracy = 99.5712336776457 %
Testing...
--> Done, accuracy = 99.

...

--> Done, accuracy = 99.30668107329372 %
Testing...
--> Done, accuracy = 99.45673517476109 %
Testing...
--> Done, accuracy = 99.65932414989737 %
Testing...
--> Done, accuracy = 99.64822776737775 %
Testing...
--> Done, accuracy = 99.82995238666827 %
Testing...
--> Done, accuracy = 99.67085402668225 %
Testing...
--> Done, accuracy = 99.46134587664062 %
Testing...
--> Done, accuracy = 99.27435883232704 %
Testing...
--> Done, accuracy = 99.8271547145352 %
Testing...
--> Done, accuracy = 99.82788598324522 %
Testing...
--> Done, accuracy = 99.65209940017138 %
Testing...
--> Done, accuracy = 99.65007450566745 %
Testing...
--> Done, accuracy = 99.6415829269091 %
Testing...
--> Done, accuracy = 99.44792050055207 %
Testing...
--> Done, accuracy = 99.6338543867061 %
Testing...
--> Done, accuracy = 99.59418070444104 %
Testing...
--> Done, accuracy = 99.8080067197648 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 100.0 %
Added Decision Tree to Random Forest!

Creating Dec

Testing...
--> Done, accuracy = 99.82245237654892 %
Testing...
--> Done, accuracy = 99.62872179367025 %
Testing...
--> Done, accuracy = 99.80834827865105 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.8088210632232 %
Testing...
--> Done, accuracy = 99.28022167204998 %
Testing...
--> Done, accuracy = 99.62803822227923 %
Testing...
--> Done, accuracy = 99.62827611085842 %
Testing...
--> Done, accuracy = 99.40469797627998 %
Testing...
--> Done, accuracy = 99.04847666216419 %
Testing...
--> Done, accuracy = 99.60609668268737 %
Testing...
--> Done, accuracy = 99.79240849395245 %
Added Decision Tree to Random Forest!

Validating...
   class  precision  recall  f1-score
0    cap     0.8644  0.9107    0.8869
1  tulis     0.9275  0.8889    0.9078
2  [avg]     0.8960  0.8998    0.8974
Training...
--> Done
Creating Decision Tree 1 ...
Testing...
--> Done, accuracy = 99.68827930174564 %
Testing...
--> Done, accuracy = 99.87021414665801 %
Testing...
--> Done, accuracy 

Testing...
--> Done, accuracy = 99.61684635073296 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.81897498293617 %
Testing...
--> Done, accuracy = 99.81123856541309 %
Testing...
--> Done, accuracy = 99.44553441951719 %
Testing...
--> Done, accuracy = 99.63643961554534 %
Testing...
--> Done, accuracy = 99.83754709769016 %
Testing...
--> Done, accuracy = 99.81668562253296 %
Added Decision Tree to Random Forest!

Creating Decision Tree 4 ...
Testing...
--> Done, accuracy = 99.25975773889637 %
Testing...
--> Done, accuracy = 99.79825151311366 %
Testing...
--> Done, accuracy = 99.70865082922457 %
Testing...
--> Done, accuracy = 99.58144985769295 %
Testing...
--> Done, accuracy = 99.65128755364807 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.61549553013553 %
Testing...
--> Done, accuracy = 99.45456071158848 %
Testing...
--> Done, accuracy = 99.64149675106431 %
Testing...
--> Done, accuracy = 99.63673057517659 %
Testing...
--> Done,

Testing...
--> Done, accuracy = 99.786968242488 %
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.65444181001867 %
Testing...
--> Done, accuracy = 99.82195845697329 %
Added Decision Tree to Random Forest!

Creating Decision Tree 2 ...
Testing...
--> Done, accuracy = 100.0 %
Testing...
--> Done, accuracy = 99.84197218710493 %
Testing...
--> Done, accuracy = 99.74710221285564 %
Testing...
--> Done, accuracy = 99.63722397476342 %
Testing...
--> Done, accuracy = 99.4994994994995 %
Testing...
--> Done, accuracy = 99.8133941530168 %
Testing...
--> Done, accuracy = 99.31652760518374 %
Testing...
--> Done, accuracy = 99.48658109684948 %
Testing...
--> Done, accuracy = 99.52219375389517 %
Testing...
--> Done, accuracy = 99.8133631952221 %
Testing...
--> Done, accuracy = 99.64914266312037 %
Testing...
--> Done, accuracy = 99.31986916567156 %
Testing...
--> Done, accuracy = 99.82294109202277 %
Testing...
--> Done, accuracy = 99.09586491158092 %
Testing...
--> Done, accu

Testing...
--> Done, accuracy = 99.81179422835633 %
Testing...
--> Done, accuracy = 99.45973112200025 %
Testing...
--> Done, accuracy = 99.87414787624542 %
Testing...
--> Done, accuracy = 99.5863681323622 %
Testing...
--> Done, accuracy = 99.4724409448819 %
Testing...
--> Done, accuracy = 99.62227196418579 %
Testing...
--> Done, accuracy = 99.14228052472251 %
Testing...
--> Done, accuracy = 99.39890084726358 %
Testing...
--> Done, accuracy = 99.60084033613445 %
Testing...
--> Done, accuracy = 99.6174519393734 %
Testing...
--> Done, accuracy = 99.79764367299218 %
Testing...
--> Done, accuracy = 99.6896624030195 %
Testing...
--> Done, accuracy = 99.85884567126725 %
Testing...
--> Done, accuracy = 99.35074516747824 %
Testing...
--> Done, accuracy = 99.46322760543744 %
Testing...
--> Done, accuracy = 99.26390493480773 %
Testing...
--> Done, accuracy = 99.31800406694823 %
Testing...
--> Done, accuracy = 99.64461699268328 %
Testing...
--> Done, accuracy = 99.44060734060163 %
Testing...
--> D